In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings 
import os
import pandas as pd
import numpy as np
# tensorflow.keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Conv1D
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D,GlobalAveragePooling1D
from tensorflow.keras.models import Model
from tensorflow.keras import initializers, regularizers, constraints, optimizers, layers
from sklearn.model_selection import train_test_split
from gensim.models import KeyedVectors
from tqdm import tqdm
import os
import gc
import time
tqdm.pandas()
import pandas as pd
import numpy as np
# tensorflow.keras
# from tensorflow.keras.preprocessing.text import Tokenizer
# from tensorflow.keras.preprocessing.sequence import pad_sequences
# from tensorflow.keras.utils import to_categorical
# from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, CuDNNGRU, Conv1D
# from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D
# from tensorflow.keras.models import Model
# from tensorflow.keras import initializers, regularizers, constraints, optimizers, layers
# from sklearn.model_selection import train_test_split
# from gensim.models import KeyedVectors
# from tqdm import tqdm
# import os
# import gc
# import time
# tqdm.pandas()

In [2]:
import nltk

In [3]:
# nltk.download()

In [4]:
df_train= pd.read_csv('input/train.csv',encoding='gbk')
df_test=pd.read_csv('input/test.csv',encoding='gbk')

In [5]:
df_train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [6]:
def clean_text(text):
    import re
    text = text.lower()
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"you'll", "you will", text)
    text = re.sub(r"i'll", "i will", text)
    text = re.sub(r"she'll", "she will", text)
    text = re.sub(r"he'll", "he will", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "what is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"there's", "there is", text)
    text = re.sub(r"here's", "here is", text)
    text = re.sub(r"who's", "who is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"don't", "do not", text)
    text = re.sub(r"shouldn't", "should not", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"   ", " ", text) # Remove any extra spaces
    return text


df_train['clean_text'] = df_train['text'].apply(clean_text)
df_test['clean_text'] = df_test['text'].apply(clean_text)

In [7]:
df_train.head()

,id,keyword,location,text,target,clean_text
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,our deeds are the reason of this #earthquake m...
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,forest fire near la ronge sask. canada
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,all residents asked to 'shelter in place' are ...
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,"13,000 people receive #wildfires evacuation or..."
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,just got sent this photo from ruby #alaska as ...


In [8]:
def massage_text(text):
    import re
    from nltk.corpus import stopwords
    ## remove anything other then characters and put everything in lowercase
    tweet = re.sub("[^a-zA-Z]", ' ', text)
    tweet = tweet.lower()
    tweet = tweet.split()

    from nltk.stem import WordNetLemmatizer
    lem = WordNetLemmatizer()
    tweet = [lem.lemmatize(word) for word in tweet
             if word not in set(stopwords.words('english'))]
    tweet = ' '.join(tweet)
    return tweet
    print('--here goes nothing')
    print(text)
    print(tweet)

df_train['clean_text'] = df_train['text'].apply(massage_text)
df_test['clean_text'] = df_test['text'].apply(massage_text)

In [9]:
df_train.head()

,id,keyword,location,text,target,clean_text
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,deed reason earthquake may allah forgive u
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,forest fire near la ronge sask canada
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,resident asked shelter place notified officer ...
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,people receive wildfire evacuation order calif...
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,got sent photo ruby alaska smoke wildfire pour...


In [10]:
from nltk.tokenize import word_tokenize

df_train['tokens']=df_train['clean_text'].apply(lambda x: word_tokenize(x))
df_test['tokens'] = df_test['clean_text'].apply(lambda x: word_tokenize(x))

In [11]:
df_train['tokens'].head()

0    [deed, reason, earthquake, may, allah, forgive...
1        [forest, fire, near, la, ronge, sask, canada]
2    [resident, asked, shelter, place, notified, of...
3    [people, receive, wildfire, evacuation, order,...
4    [got, sent, photo, ruby, alaska, smoke, wildfi...
Name: tokens, dtype: object

In [12]:
maxlen = 250
data = pd.concat([df_train,df_test]).reset_index(drop=True)

In [13]:
max_features =7965
tokenizer = Tokenizer(num_words=max_features,lower=True)
tokenizer.fit_on_texts(list(data['tokens']))
#word_index = tokenizer.word_index
x_train = tokenizer.texts_to_sequences(list(df_train['tokens']))
x_train = pad_sequences(x_train,maxlen=maxlen) # padding

In [14]:
x_train[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [15]:
y_train=df_train['target']

In [16]:
x_train,x_val,y_train,y_val = train_test_split(x_train ,y_train,test_size=0.3,random_state=0)

In [17]:
df_train['clean_text'].head()

0           deed reason earthquake may allah forgive u
1                forest fire near la ronge sask canada
2    resident asked shelter place notified officer ...
3    people receive wildfire evacuation order calif...
4    got sent photo ruby alaska smoke wildfire pour...
Name: clean_text, dtype: object

In [18]:
import gensim
def fn_pre_process_data(doc):
    for rec in doc:
        yield gensim.utils.simple_preprocess(rec)

corpus = list(fn_pre_process_data(df_train['clean_text']))
corpus += list(fn_pre_process_data(df_test['clean_text']))

In [19]:
from gensim.models import Word2Vec

print('initiated ...')
wv_model = Word2Vec(corpus,size=150,window=3,min_count=2)
wv_model.train(corpus,total_examples=len(corpus),epochs=10)

initiated ...


(915442, 1226180)

In [20]:
w2v_weights = wv_model.wv.vectors

In [21]:
embed_size=150

In [22]:
max_features=7965

In [23]:
def build_model(embedding_matrix=None):
    inp = Input(shape=(maxlen,))
    if embedding_matrix is None:
        x = Embedding(max_features, embed_size)(inp)
    else:
        x = Embedding(max_features, embed_size, weights=[w2v_weights])(inp)
    x = Bidirectional(LSTM(64, return_sequences=True))(x)
    x = Bidirectional(LSTM(64, return_sequences=True))(x)
    x = GlobalMaxPool1D()(x)
#     x = GlobalMeanPool1D()(x)
    x = Dense(128, activation="relu")(x)   
    x = Dense(128, activation="relu")(x)  
    x = Dropout(0.1)(x)
    x = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [24]:
from tensorflow import keras

In [25]:

glove_embedding_matrix=1

model = build_model(embedding_matrix=1)


In [26]:
x_train[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   

In [27]:
history = model.fit(x_train, y_train, batch_size=512, epochs=5, validation_data=(x_val, y_val))

Epoch 1/5
11/11 [==============================] - 82s 7s/step - loss: 0.6751 - accuracy: 0.5609 - val_loss: 0.6422 - val_accuracy: 0.6511
Epoch 2/5
11/11 [==============================] - 87s 8s/step - loss: 0.6385 - accuracy: 0.6337 - val_loss: 0.6172 - val_accuracy: 0.6659
Epoch 3/5
11/11 [==============================] - 95s 9s/step - loss: 0.6205 - accuracy: 0.6579 - val_loss: 0.6164 - val_accuracy: 0.6629
Epoch 4/5
11/11 [==============================] - 118s 11s/step - loss: 0.5657 - accuracy: 0.7206 - val_loss: 0.5328 - val_accuracy: 0.7452
Epoch 5/5
11/11 [==============================] - 111s 10s/step - loss: 0.4427 - accuracy: 0.8067 - val_loss: 0.4641 - val_accuracy: 0.7977


In [28]:
from tensorflow.keras.layers import concatenate

In [29]:
def build_model2(embedding_matrix=None):
    inp = Input(shape=(maxlen,))
    if embedding_matrix is None:
        x = Embedding(max_features, embed_size)(inp)
    else:
        x = Embedding(max_features, embed_size, weights=[w2v_weights])(inp)
    x = Bidirectional(LSTM(128, return_sequences=True))(x)
    x = Bidirectional(LSTM(128, return_sequences=True))(x)
    x1 = GlobalMaxPool1D()(x)
#     x = GlobalMeanPool1D()(x)
    x2 = GlobalAveragePooling1D()(x)
    x = concatenate([x1,x2])
    x = Dense(128, activation="relu")(x)   
    x = Dropout(0.5)(x)
    x = Dense(128, activation="relu")(x) 
    x = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [30]:
model = build_model2(embedding_matrix=1)

In [31]:
model.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 250)]        0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 250, 150)     1194750     input_2[0][0]                    
__________________________________________________________________________________________________
bidirectional_2 (Bidirectional) (None, 250, 256)     285696      embedding_1[0][0]                
__________________________________________________________________________________________________
bidirectional_3 (Bidirectional) (None, 250, 256)     394240      bidirectional_2[0][0]            
_______________________________________________________________________________________

In [32]:
history = model.fit(x_train, y_train, batch_size=512, epochs=5, validation_data=(x_val, y_val))

Epoch 1/5
11/11 [==============================] - 249s 23s/step - loss: 0.6900 - accuracy: 0.5451 - val_loss: 0.6694 - val_accuracy: 0.5858
Epoch 2/5
11/11 [==============================] - 287s 26s/step - loss: 0.6688 - accuracy: 0.5810 - val_loss: 0.6318 - val_accuracy: 0.6493
Epoch 3/5
11/11 [==============================] - 237s 22s/step - loss: 0.6456 - accuracy: 0.6161 - val_loss: 0.6238 - val_accuracy: 0.6537
Epoch 4/5
11/11 [==============================] - 246s 22s/step - loss: 0.6357 - accuracy: 0.6448 - val_loss: 0.6101 - val_accuracy: 0.6721
Epoch 5/5
11/11 [==============================] - 253s 23s/step - loss: 0.6127 - accuracy: 0.6675 - val_loss: 0.5756 - val_accuracy: 0.7049
